In [3]:
##### 필요한 모듈/라이브러리 설치 #####

from selenium import webdriver
from bs4 import BeautifulSoup 
import pandas as pd
import sys, time, math, os, random
import urllib.request
import urllib
import cx_Oracle

##### 드라이버 설치 #####

path = "c:/temp/chromedriver_85/chromedriver.exe"
driver=webdriver.Chrome(path)

##### 검색어 입력 #####  

print("=" *80)
print("강아지 수제간식 네이버 쇼핑 검색하기")
print("=" *80)

query_text = "강아지 수제간식"

query_ingredients = int(input('''
  0. 전체       1. 오리       2. 고구마        3. 과일/야채   4. 북어       
  5. 돼지       6. 양         7. 닭            8. 연어        9. 치즈/유지방       
 10. 사슴      11.칠면조     12.생선/해산물   13.소        

1. 위 분야 중에서 자료를 수집할 번호를 선택하세요 :'''))
print("=" *80)

query_menu = int(input('''
1. 네이버 랭킹순 2. 낮은 가격순 3. 높은 가격순 4. 등록일순 5. 리뷰 많은순 

2. 분류할 방법을 입력해주세요 : '''))
print("=" *80)

cnt = int(input('3.크롤링 할 건수는 몇 건입니까?: '))
print("=" *80)

f_dir = input("4.파일을 저장할 폴더명만 쓰세요(기본경로:c:\\temp\\):")
if f_dir == '' :
    f_dir = "c:\\temp\\"

##### 파일 저장 위치 설정 #####   

# 현재 시간 저장 
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' \
% (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

# txt, csv, xls 파일 저장 위치 및 파일명 지정 
os.makedirs(f_dir+s+'-'+query_text)
os.chdir(f_dir+s+'-'+query_text)

f_name=f_dir+s+'-'+query_text+'\\'+s+'-'+query_text+'.txt'    # 메모장 파일 
fc_name=f_dir+s+'-'+query_text+'\\'+s+'-'+query_text+'.csv'   # 쉼표 있는 엑셀 
fx_name=f_dir+s+'-'+query_text+'\\'+s+'-'+query_text+'.xls'   # 엑셀 

# 이미지 저장 위치 및 파일명 지정 
img_dir = f_dir+s+'-'+query_text+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

##### 웹페이지 접속 후 정보 추출 #####  

# 강아지 수제간식 키워드로 네이버 쇼핑 접속 
driver.get('https://shopping.naver.com/home/p/index.nhn')
driver.maximize_window()
element = driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/input[1]').send_keys(query_text)
driver.find_element_by_xpath('//*[@id="autocompleteWrapper"]/a[2]').click()

# 간식 재료 선택하기 
if query_ingredients == 0 :
    driver.get('https://search.shopping.naver.com/search/all?frm=NVSHCHK&origQuery=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%88%98%EC%A0%9C%EA%B0%84%EC%8B%9D&pagingIndex=1&pagingSize=40&productSet=checkout&query=%EA%B0%95%EC%95%84%EC%A7%80%20%EC%88%98%EC%A0%9C%EA%B0%84%EC%8B%9D&sort=rel&timestamp=&viewType=list')
else :
    driver.find_element_by_xpath\
    ('//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[3]/div[2]/div/ul/li[{}]/a/span[1]'\
    .format(query_ingredients)).click()

# 검색 순서 선택하기 
if query_menu ==1 :
    driver.find_element_by_class_name('subFilter_sort__rQUtM').click 
elif query_menu ==2 :
    driver.find_element_by_link_text('낮은 가격순').click()
elif query_menu ==3 :
    driver.find_element_by_link_text('높은 가격순').click()
elif query_menu ==4 :
    driver.find_element_by_link_text('등록일순').click()
else : 
    driver.find_element_by_link_text('리뷰 많은순').click()
time.sleep(random.randrange(2,5))

# 스크롤바를 끝까지 내리기 
last_page_height = driver.execute_script("return document.documentElement.scrollHeight") 
while True: 
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
    time.sleep(3.0) 
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")     
    if new_page_height == last_page_height: 
        break 
    last_page_height = new_page_height

# 메인 페이지 정보에서 가능한 검색 숫자 추출하여, 실제 검색 숫자와 맞추기 
html=driver.page_source
soup = BeautifulSoup(html,'html.parser')
search_cnt=int(soup.find('ul','subFilter_seller_filter__3yvWP')\
.find_all('span')[0].get_text().strip().replace(",",""))
time.sleep(3)

if cnt > int(search_cnt) :       # 입력 건수가 실제 건수보다 크면 
    cnt1 = int(search_cnt)       # 실제 건수로 출력하라 
else:
    cnt1 = cnt                   # 아닐 경우, 실제 건수로 출력하라 
    
real_page_cnt=math.ceil(cnt1/40) # 한 페이지에 포함된 개별 정보의 숫자로 나눠 페이지 계산 

# 사용자 화면 출력 
print("\n")
print("="*80)
print("요청하신 검색 건수는 {}건이고,".format(cnt))
print("해당 키워드로 실제 검색된 글의 건수는 총 {}건이라서,".format(search_cnt))
print("실제 검색 건수는 {}건으로 재설정하고,".format(cnt1))
print("총 {}페이지의 게시글 수집을 진행하겠습니다".format(real_page_cnt))
print("단, 가격비교, 판매중단, 개별쇼핑몰 정보는 수집되지 않으므로, 실제 수집 건수는 다를 수 있습니다.")
print("="*80)
print("\n")
time.sleep(2)

##### 필요 변수 생성 #####    

no=0                # 번호 시작점 설정 
no2=[]              # 1. 번호 
products2=[]        # 2. 상품명  
provider2=[]        # 3. 판매처
price2=[]           # 4. 정가 
off_price2=[]       # 5. 할인가 
review2=[]          # 6. 리뷰 수
rating2=[]          # 7. 평점
made_in2=[]         # 8. 원산지 
img_store2=[]       # 9. 이미지 저장위치 
file_no = 0         # 이미지 파일번호 시작점 설정

##### 페이지 크롤링 함수 + .txt 파일 저장 / 네이버 스토어 #####  

def smartstore_crawling():

    global no 
    global file_no 

    f = open(f_name, 'a', encoding='UTF-8')
    f.write("-----------------------------------------------------"+"\n")
    print("-" *70)

    # 1. 번호 
    no+=1
    print('1. 번호:',no)
    f.write('1. 번호:'+str(no)+"\n")

    # 2. 상품명 
    try : 
        products = soup2.find('h3','_3oDjSvLwq9').get_text().strip()   
    except : 
        products = '상품명 없음'
        print('2. 상품명:', products)   
        f.write('2. 상품명:'+str(products)+"\n")    
    else : 
        print('2. 상품명:', products)
        f.write('2. 상품명:'+str(products)+"\n") 

    # 3. 판매처 
        try : 
            provider = soup2.find('span','_3QBCSk9AvE').get_text().strip()
        except : 
            provider = '판매처 없음'
            print('3. 판매처:', provider)
            f.write('3. 판매처:'+str(provider)+"\n") 
        else : 
            print('3. 판매처:', provider)
            f.write('3. 판매처:'+str(provider)+"\n") 

    # 4. 정가 
    try :  
        price = int(soup2.find('span','_1LY7DqCnwR').get_text().replace(",",""))  
    except : 
        price = int(soup2.find('del','Xdhdpm0BD9').find_all('span')[1].get_text().replace(",",""))
        print('4. 정가:', price)
        f.write('4. 정가:'+str(price)+"\n") 
    else : 
        print('4. 정가:', price)
        f.write('4. 정가:'+str(price)+"\n")

    # 4. 할인가 
    try : 
        off_price = int(soup2.find('strong','aICRqgP9zw').find_all('span')[1].get_text().replace(",",""))
    except : 
        off_price = int(0)
        print('5. 할인가:', off_price)
        f.write('5. 할인가:'+str(off_price)+"\n")
    else : 
        print('5. 할인가:', off_price)
        f.write('5. 할인가:'+str(off_price)+"\n")

    # 6. 리뷰수 
    try : 
        review = int(soup2.find('div','NFNlCQC2mv').find_all('strong')[0].get_text().replace(",",""))
    except : 
        review = int(soup2.find('div','NFNlCQC2mv').find('a').find('strong').get_text().replace(",",""))
        print('6. 리뷰 수:', review)
        f.write('6. 리뷰 수:'+str(review)+"\n")
    else : 
        print('6. 리뷰 수:', review)
        f.write('6. 리뷰 수:'+str(review)+"\n")

    # 7. 리뷰 평점 
    rating = float(soup2.find('div','NFNlCQC2mv').find_all('strong')[1].get_text().split('/')[0])
    print('7. 리뷰 평점:', rating)
    f.write('7. 리뷰 평점:'+str(rating)+"\n")
    
    # 8. 원산지
    info_tr = len(soup2.find('tbody').find_all('tr'))
    info_td = len(soup2.find('tbody').find_all('td'))
    
    if info_tr ==2 and info_td==3 :
        made_in = soup2.find('tbody').find_all('tr')[1].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")      
    elif info_tr ==2 and info_td==4 :
        made_in = soup2.find('tbody').find_all('tr')[1].find_all('td')[1].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")      
    elif info_tr ==3 and info_td==5 :
        made_in = soup2.find('tbody').find_all('tr')[2].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==3 and info_td==6 :
        made_in = soup2.find('tbody').find_all('tr')[2].find_all('td')[1].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==4 and info_td==7 :
        made_in = soup2.find('tbody').find_all('tr')[3].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==4 and info_td==8 :
        made_in = soup2.find('tbody').find_all('tr')[3].find_all('td')[1].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==5 and info_td==9 :
        made_in = soup2.find('tbody').find_all('tr')[3].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    else :
        made_in = soup2.find('tbody').find_all('tr')[3].find_all('td')[1].get_text()
        print('9. 원산지:', made_in)
        f.write('9. 원산지:'+str(made_in)+"\n")
    
    # 9. 제품 이미지 
    try : 
        photo = soup2.find('div','_23RpOU6xpc').find('img')['src']
    except : 
        pass 
    
    file_no += 1
    urllib.request.urlretrieve(photo,str(file_no)+'.jpg')

    image_name = str(file_no) + '.jpg'
    img_store = f_dir+s+'-'+query_text+"\\images"+"\\"+image_name
    print ('8. 이미지 경로:',img_store)
    f.write('8. 이미지 경로:'+str(img_store)+"\n")
    print('\n')
    time.sleep(2)

    no2.append(no)                 # 1. 번호 
    products2.append(products)     # 2. 상품명 
    provider2.append(provider)     # 3. 판매처 
    price2.append(price)           # 4. 정가 
    off_price2.append(off_price)   # 5. 할인가 
    review2.append(review)         # 6. 리뷰수 
    rating2.append(rating)         # 7. 평점 
    made_in2.append(made_in)       # 8. 원산지 
    img_store2.append(img_store)   # 9. 이미지 저장위치 

##### 페이지 크롤링 함수 + .txt 파일 저장 / 펫 윈도 ##### 

def pet_window_crawling():

    global no 
    global file_no 

    f = open(f_name, 'a', encoding='UTF-8')
    f.write("-----------------------------------------------------"+"\n")
    print("-" *70)

    # 1. 번호 
    no+=1
    print('1. 번호:',no)
    f.write('1. 번호:'+str(no)+"\n") 

    # 2. 상품명 
    try : 
        products = soup2.find('h3','_2IA5sp7BRM').get_text().strip()       
    except : 
        products = '상품명 없음'
        print('2. 상품명:', products) 
        f.write('2. 상품명:'+str(products)+"\n")           
    else : 
        print('2. 상품명:', products)
        f.write('2. 상품명:'+str(products)+"\n") 

    # 3. 판매처 
    try : 
        provider = soup2.find('span','_3QBCSk9AvE').get_text().strip()
    except : 
        provider = '판매처 없음'
        print('3. 판매처:', provider)
        f.write('3. 판매처:'+str(provider)+"\n") 
    else : 
        print('3. 판매처:', provider)
        f.write('3. 판매처:'+str(provider)+"\n") 

    # 4. 정가 
    try :      
        price = int(soup2.find('span','_1LY7DqCnwR').get_text().replace(",",""))     
    except : 
        price = int(soup2.find('del','Xdhdpm0BD9').find_all('span')[1].get_text().replace(",",""))
        print('4. 정가:', price)
        f.write('4. 정가:'+str(price)+"\n")
    else : 
        print('4. 정가:', price)
        f.write('4. 정가:'+str(price)+"\n")

    # 5. 할인가 
    try : 
        off_price = int(soup2.find('strong','aICRqgP9zw').find_all('span')[1]\
        .get_text().replace(",",""))
    except : 
        off_price = int(0)
        print('5. 할인가:', off_price)
        f.write('5. 할인가:'+str(off_price)+"\n")
    else : 
        print('5. 할인가:', off_price)
        f.write('5. 할인가:'+str(off_price)+"\n")

    # 6. 리뷰 수 
    review = int(soup2.find('div','NFNlCQC2mv').find_all('strong')[0].get_text().replace(",",""))
    print('6. 리뷰 수:', review)
    f.write('6. 리뷰 수:'+str(review)+"\n")

    # 7. 리뷰 평점 
    rating = float(soup2.find('div','NFNlCQC2mv').find_all('strong')[1].get_text().split("/")[0])
    print('7. 리뷰 평점:', rating)
    f.write('7. 리뷰 평점:'+str(rating)+"\n")
  
    # 8. 원산지 
    info_tr = int(len(soup2.find('table','TH_yvPweZa').find_all('tr')))
    info_td = int(len(soup2.find('table','TH_yvPweZa').find_all('td')))

    if info_tr ==2 and info_td==3 :
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[1].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==2 and info_td==4 :
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[1].find_all('td')[1].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==3 and info_td==5 :
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[2].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==3 and info_td==6 :
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[2].find_all('td')[1].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==4 and info_td==7 :
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[3].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==4 and info_td==8 :
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[3].find_all('td')[1].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    elif info_tr ==5 and info_td==9 :
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[3].find_all('td')[0].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
    else : 
        made_in = soup2.find('table','TH_yvPweZa').find_all('tr')[3].find_all('td')[1].get_text()
        print('8. 원산지:', made_in)
        f.write('8. 원산지:'+str(made_in)+"\n")
        
    # 9. 제품 이미지 
    try : 
        photo = soup2.find('div','_23RpOU6xpc').find('img')['src']
    except :
        pass 

    file_no += 1
    urllib.request.urlretrieve(photo,str(file_no)+'.jpg')

    image_name = str(file_no) + '.jpg'
    img_store = f_dir+s+'-'+query_text+"\\images"+"\\"+image_name
    print ('9. 이미지 경로:',img_store)
    f.write('9. 이미지 경로:'+str(img_store)+"\n")
    print('\n')                
    time.sleep(2)
                
    no2.append(no)                 # 1. 번호 
    products2.append(products)     # 2. 상품명 
    provider2.append(provider)     # 3. 판매처 
    price2.append(price)           # 4. 정가 
    off_price2.append(off_price)   # 5. 할인가 
    review2.append(review)         # 6. 리뷰수 
    rating2.append(rating)         # 7. 평점 
    made_in2.append(made_in)       # 8. 원산지 
    img_store2.append(img_store)   # 9. 이미지 저장위치 

##### 전체 페이지 크롤링 #####  

s_time = time.time()

for x in range(1, real_page_cnt+1) :  

    print("======= {} 페이지 내용 수집 시작합니다 =======" .format(x) , "\n")
    time.sleep(1)

    for i in range (1,40+1):
 
        driver.find_element_by_xpath \
        ('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/ul/div/div[{}]/li/div/div[2]/div[1]/a'\
        .format(i)).click()
        time.sleep(2)

        driver.switch_to_window(driver.window_handles[1])
        html=driver.page_source
        soup2 = BeautifulSoup(html,'html.parser')    

        smartstore = soup2.find ('body','smartstore')          # 스마트스토어 고유 속성 
        pet_window = soup2.find ('div','_2OVEkKuV0W')          # 펫윈도 고유 속성
        low_price = soup2.find('span','low')                   # 최저가 가격비교 고유 속성
        smartstore_shutdown = soup2.find('div','_2BQ-WF2QUb')  # 스마트스토어 판매 중단 고유 속성 
        pet_window_shutdown = soup2.find('h2','CUJdaH2pd6')    # 펫윈도 판매 중단 고유 속성 

        if smartstore : 
            smartstore_crawling ()
        elif pet_window :
            pet_window_crawling()
        elif smartstore_shutdown : 
            # no+=1
            # print('0. 번호:',no)
            # no2.append(no)
            print ('판매 중단 정보 수집 없음')
            print('\n')
        elif pet_window_shutdown : 
            # no+=1
            # print('0. 번호:',no)
            # no2.append(no)
            print ('판매 중단 정보 수집 없음')
            print('\n')
        elif low_price : 
            # no+=1
            # print('0. 번호:',no)
            # no2.append(no)
            print ('최저가 쇼핑몰 정보 수집 없음')           
            print('\n')
        else :                                       
            # no+=1
            # print('0. 번호:',no)
            # no2.append(no)
            print ('개별 쇼핑몰 정보 수집 없음')
            print('\n')

        driver.close()
        driver.switch_to_window(driver.window_handles[0])  
        time.sleep(2)

        if no == cnt1 :
            break  
    
    x += 1

    if x <= 2 :
        driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[3]/a').click()
    else :
        driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[3]/a[2]').click()
    time.sleep(2)

##### 크롤링 결과 처리 ##### 
                
# 데이터 프레임으로 변환 
naver_shopping= pd.DataFrame()
naver_shopping['번호'] = pd.Series(no2)    
naver_shopping['상품명'] = pd.Series(products2)
naver_shopping['판매처'] = pd.Series(provider2)
naver_shopping['정가'] = pd.Series(price2)
naver_shopping['할인가'] = pd.Series(off_price2)
naver_shopping['리뷰수'] = pd.Series(review2)
naver_shopping['평점'] = pd.Series(rating2)
naver_shopping['원산지'] = pd.Series(made_in2)
naver_shopping['이미지']= pd.Series(img_store2)

# csv, excel 파일로 저장 
naver_shopping.to_csv(fc_name, encoding="utf-8-sig", index=False)
naver_shopping.to_excel(fx_name, index=False)

# 사용자 화면 출력 
print("\n" + "="*80)
print('''
요청한 총 {0} 건 중에서 {1} 건의 데이터를 수집 완료. 총 {2} 초 소요 \n
파일 저장(txt): {3} 
파일 저장(csv): {4} 
파일 저장(xls): {5} 
''' .format(cnt, cnt1, round(time.time() - s_time), f_name, fc_name, fx_name))
print("="*80 + "\n")

driver.close()

#### 오라클 연결 ####
                
# 오라클 계정 연결 
con = cx_Oracle.connect('naver_crawler/shopping00@localhost:1521/testdb')
cursor = con.cursor()

# 데이터 프레임을 리스트로 변환
dataframe_list=naver_shopping.values.tolist()  
sql='''INSERT INTO naver_shopping
            (no, products, provider, price, off_price, review, rating, made_in, img_store)
            values (:1,:2,:3,:4,:5,:6,:7,:8,:9)'''
print(sql)
# 리스트를 반복해서 삽입                
for index, i in enumerate(dataframe_list):  
    cursor.execute(sql,dataframe_list[index]) 

con.commit()
cursor.close()
con.close()


강아지 수제간식 네이버 쇼핑 검색하기

  0. 전체       1. 오리       2. 고구마        3. 과일/야채   4. 북어       
  5. 돼지       6. 양         7. 닭            8. 연어        9. 치즈/유지방       
 10. 사슴      11.칠면조     12.생선/해산물   13.소        

1. 위 분야 중에서 자료를 수집할 번호를 선택하세요 :4

1. 네이버 랭킹순 2. 낮은 가격순 3. 높은 가격순 4. 등록일순 5. 리뷰 많은순 

2. 분류할 방법을 입력해주세요 : 5
3.크롤링 할 건수는 몇 건입니까?: 300
4.파일을 저장할 폴더명만 쓰세요(기본경로:c:\temp\):c:\temp\


요청하신 검색 건수는 300건이고,
해당 키워드로 실제 검색된 글의 건수는 총 2121건이라서,
실제 검색 건수는 300건으로 재설정하고,
총 8페이지의 게시글 수집을 진행하겠습니다
단, 가격비교, 판매중단, 개별쇼핑몰 정보는 수집되지 않으므로, 실제 수집 건수는 다를 수 있습니다.


======= 1 페이지 내용 수집 시작합니다 ======= 



c:\python354\lib\site-packages\ipykernel_launcher.py:433: DeprecationWarning: use driver.switch_to.window instead


----------------------------------------------------------------------
1. 번호: 1
2. 상품명: 상품명 없음
3. 판매처: 네이월
4. 정가: 6300
5. 할인가: 6200
6. 리뷰 수: 3193
7. 리뷰 평점: 4.9
8. 원산지: 러시아산
9. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\1.jpg




c:\python354\lib\site-packages\ipykernel_launcher.py:473: DeprecationWarning: use driver.switch_to.window instead


----------------------------------------------------------------------
1. 번호: 2
2. 상품명: 네이월 무염황태 노견 아기강아지추천  노알러지 수제간식 40g
3. 판매처: 네이월
4. 정가: 6300
5. 할인가: 6200
6. 리뷰 수: 3193
7. 리뷰 평점: 4.9
8. 원산지: 러시아산
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\2.jpg


----------------------------------------------------------------------
1. 번호: 3
2. 상품명: 취향대로 골라먹는~ 동결건조 푹익힌 야채 단품
3. 판매처: 도란도란 수제간식
4. 정가: 5000
5. 할인가: 5000
6. 리뷰 수: 2815
7. 리뷰 평점: 4.9
8. 원산지: 비트, 고구마, 연근, 당근 - 국내산  / 단호박 - 뉴질랜드
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\3.jpg


----------------------------------------------------------------------
1. 번호: 4
2. 상품명: 펫파더 동결건조 간식
3. 판매처: 펫 파더
4. 정가: 38000
5. 할인가: 7000
6. 리뷰 수: 2791
7. 리뷰 평점: 4.9
8. 원산지: 오리,치즈,치킨,소간,말고기(국내산) 명태(러시아산)소고기(호주)연어(미국)
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\4.jpg


----------------------------------------------------------------------
1. 번호: 5
2. 상품명: 후코홀릭 수제간식 후코이단/ 후코명태/ 북어볼 20종
3. 판매처: 흰둥이네
4. 정가: 13000
5. 할인가: 2190
6. 리뷰 

...

9. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\192.jpg


======= 6 페이지 내용 수집 시작합니다 ======= 

----------------------------------------------------------------------
1. 번호: 193
2. 상품명: 파우더마약팩 200g 수제간식 가루 댕댕이냥냥
3. 판매처: 댕댕이냥냥
4. 정가: 15000
5. 할인가: 14900
6. 리뷰 수: 45
7. 리뷰 평점: 4.8
8. 원산지: 국산(경상남도 합천군)
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\193.jpg


----------------------------------------------------------------------
1. 번호: 194
2. 상품명: [펫츠맘마] 즐거운식사 국내산 애견수제간식 50-60g
3. 판매처: 펫츠맘마
4. 정가: 3600
5. 할인가: 2100
6. 리뷰 수: 45
7. 리뷰 평점: 4.9
8. 원산지: 상세설명에 표시
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\194.jpg


----------------------------------------------------------------------
1. 번호: 195
2. 상품명: 황태 육포
3. 판매처: 모카맘 간식나라
4. 정가: 5000
5. 할인가: 5000
6. 리뷰 수: 44
7. 리뷰 평점: 4.8
8. 원산지: 국산
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\195.jpg


----------------------------------------------------------------------
1. 번호: 196
2. 상품명: 상품명 없음
3. 판매처: 개밥의신
4. 정가: 3690
5

----------------------------------------------------------------------
1. 번호: 225
2. 상품명: 강아지새우튀김 - 오간식
3. 판매처: 오간식
4. 정가: 7500
5. 할인가: 7500
6. 리뷰 수: 37
7. 리뷰 평점: 4.8
8. 원산지: 국산
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\225.jpg


----------------------------------------------------------------------
1. 번호: 226
2. 상품명: 시금치 스무디 강아지 수제간식 수제츄르 영양식
3. 판매처: 밤시 수제간식 :BAMSI
4. 정가: 5500
5. 할인가: 5500
6. 리뷰 수: 37
7. 리뷰 평점: 5.0
8. 원산지: 국산
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\226.jpg


----------------------------------------------------------------------
1. 번호: 227
2. 상품명: 냐무환+인트라젠 5.5 kg 이상 (테라코트환)
3. 판매처: 냐무냐무
4. 정가: 18000
5. 할인가: 18000
6. 리뷰 수: 36
7. 리뷰 평점: 4.9
8. 원산지: 국산(대전광역시 대덕구)
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\227.jpg


----------------------------------------------------------------------
1. 번호: 228
2. 상품명: 네츄럴이엑스 황태츄 꽈배기 M
3. 판매처: 네츄럴EX
4. 정가: 2000
5. 할인가: 2000
6. 리뷰 수: 36
7. 리뷰 평점: 4.8
8. 원산지: 국산(경상남도 김해시)
8. 이미지 경로: c:\temp\2020-09-2

----------------------------------------------------------------------
1. 번호: 257
2. 상품명: 아침애 수제간식 북어와 소고기60g
3. 판매처: 펫타곤
4. 정가: 3000
5. 할인가: 3000
6. 리뷰 수: 31
7. 리뷰 평점: 5.0
8. 원산지: 국산
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\257.jpg


----------------------------------------------------------------------
1. 번호: 258
2. 상품명: 상품명 없음
3. 판매처: 수제로만
4. 정가: 15900
5. 할인가: 11900
6. 리뷰 수: 32
7. 리뷰 평점: 4.9
8. 원산지: 국산
9. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\258.jpg


----------------------------------------------------------------------
1. 번호: 259
2. 상품명: 상품명 없음
3. 판매처: 해피팡팡 수제간식
4. 정가: 6500
5. 할인가: 6500
6. 리뷰 수: 31
7. 리뷰 평점: 4.7
8. 원산지: 국산
9. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\259.jpg


----------------------------------------------------------------------
1. 번호: 260
2. 상품명: 상품명 없음
3. 판매처: 펫츠맘마
4. 정가: 10600
5. 할인가: 6900
6. 리뷰 수: 31
7. 리뷰 평점: 4.5
8. 원산지: 2018.01.01.
9. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\260.jpg


-----------------------

6. 리뷰 수: 25
7. 리뷰 평점: 4.9
8. 원산지: 상세설명에 표시
9. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\289.jpg


----------------------------------------------------------------------
1. 번호: 290
2. 상품명: 통큰명가 통큰수제 간식 22종 택1
3. 판매처: 라임펫
4. 정가: 5700
5. 할인가: 5700
6. 리뷰 수: 25
7. 리뷰 평점: 4.9
8. 원산지: 국산(경상남도 김해시)
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\290.jpg


----------------------------------------------------------------------
1. 번호: 291
2. 상품명: 황태 수제파우더 70g
3. 판매처: 아침애견쇼핑
4. 정가: 7000
5. 할인가: 4200
6. 리뷰 수: 25
7. 리뷰 평점: 4.8
8. 원산지: 국산
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\291.jpg


----------------------------------------------------------------------
1. 번호: 292
2. 상품명: 네츄럴코어 오리 황태죽/ 오리단호박죽 /흑염소 들깨죽 120g
3. 판매처: 도그왕
4. 정가: 1600
5. 할인가: 1600
6. 리뷰 수: 24
7. 리뷰 평점: 4.7
8. 원산지: 국산
8. 이미지 경로: c:\temp\2020-09-25-18-16-27-강아지 수제간식\images\292.jpg


----------------------------------------------------------------------
1. 번호: 293
2. 상품명: 북어 오리 크로와상 200g
3. 판매처: 라임펫
4. 정가:

IntegrityError: ORA-00001: 무결성 제약 조건(NAVER_CRAWLER.NAVER)에 위배됩니다